In [1]:
import pandas as pd
import re

In [2]:
# read start line from last_line.txt
with open('last_line.txt', 'r') as f:
    START_LINE = int(f.read())
CURR_DATE = '24/11/08-11'

In [3]:
file = 'import.xlsx'

In [4]:
df = pd.read_excel(file, dtype={'Strona transakcji': str})

c:\Users\Kamil Tomaszek\Desktop\OREX\repo\banki\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
def get_acc_number(rachunek):
    # Define the mapping dictionary
    mapping = {
        '2248': '001', '2475': '002', '1123': '003', '5899': '004', '1107': '005',
        '2497': '006', '2516': '007', '0668': '009', '5898': '010', '7200': '011',
        '0460': '012', '2474': '013', '6916': '018', '5195': '019', '0002': '020',
        '0001': '021', '0007': '022'
    }
    
    # Extract the last 4 digits
    last_four_digits = rachunek[-4:]
    
    # Get the corresponding 3-digit code
    code = mapping.get(last_four_digits, 'Unknown')
    
    # Construct the Journal Batch Name and Bal. Account No.
    journal_batch_name = f'IMPORT-{code}'
    bal_account_no = code
    
    return journal_batch_name, bal_account_no

In [6]:
REZ = r'(^|\b|\D|00000)(1( *\d){6})(\D|\b)'

def find_reservation(text):
    rezerwacje = re.findall(REZ, text)
    
    if len(rezerwacje) == 1 or (len(rezerwacje) == 2 and rezerwacje[0][1].replace(' ', '') == rezerwacje[1][1].replace(' ', '')):
        return rezerwacje[0][1].replace(' ', '')
    else:
        return ''

In [7]:
def get_payment_side(row):
    if row['Nr konta przeciwst.'] == '010':
        if row['Reservation No.'] != '':
            if row['Kwota'] < 0:
                result = ('Nabywca', 'Płatność')
            result = ('Nabywca', 'Zwrot')
        elif row['Kwota'] < 0:
                result = ('Dostawca', 'Zwrot')
        else:
            result = ('Dostawca', 'Płatność')
    elif row['Nr konta przeciwst.'] == '009':
        result = ('Konto K/G', 'Płatność')
    else:
        if row['Kwota'] < 0:
            result = ('Nabywca', 'Płatność')
        else:
            result = ('Dostawca', 'Płatność')
    return result

In [8]:
# df2 = pd.DataFrame(index=df.index, columns=[
#     'Journal Template Name', 'Journal Batch Name', 'Line No.', 'Posting Date', 
#     'Document Type', 'Account Type', 'Document No.', 'External Document No.', 
#     'Reservation No.', 'Amount', 'Message to Recipient', 'Payer Information', 
#     'Bal. Account Type', 'Bal. Account No.'
# ])
# df2['Journal Template Name'] = 'PŁATNOŚCI'
# df2['Journal Batch Name'] = df['Rachunek'].apply(lambda x: get_acc_number(x)[0])
# df2['Line No.'] = [START_LINE + i + 1 for i in range(len(df))]
# df2['Posting Date'] = df['Data księgowania']
# df2['Document No.'] = [f'imp/{CURR_DATE}/{i+1:04d}' for i in range(len(df))]
# df2['External Document No.'] = df2['Document No.']
# df2['Reservation No.'] = df['Tytuł'].apply(find_reservation).replace(' ', '')
# df2['Amount'] = -df['Kwota']
# df2['Message to Recipient'] = df['Tytuł']
# df2['Payer Information'] = df['Strona transakcji'].apply(lambda x: x[:50] if isinstance(x, str) else ' ')
# df2['Bal. Account Type'] = 'Bank Account'
# df2['Bal. Account No.'] = df['Rachunek'].apply(lambda x: get_acc_number(x)[1])
# df2[['Account Type', 'Document Type']] = df2.apply(lambda row: pd.Series(get_payment_side(row)), axis=1)

In [ ]:
df2 = pd.DataFrame(index=df.index, columns=[
    'Nazwa szablonu dziennika', 'Nazwa instancji dziennika', 'Nr wiersza', 'Typ konta', 'Nr konta', 'Data księgowania',
    'Typ dokumentu', 'Nr dokumentu', 'Typ konta przeciwst.', 'Nr konta przeciwst.', 'Nr dokumentu zewnętrznego', 'Kwota',
    'Wiadomość dla odbiorcy', 'Opis', 'Informacje o płatniku', 'Reservation No.'
])

df2['Nazwa szablonu dziennika'] = 'PŁATNOŚCI'
df2['Nazwa instancji dziennika'] = df['Rachunek'].apply(lambda x: get_acc_number(x)[0])
df2['Nr wiersza'] = [START_LINE + i + 1 for i in range(len(df))]
df2['Data księgowania'] = df['Data księgowania']
df2['Nr dokumentu'] = [f'imp/{CURR_DATE}/{i+1:04d}' for i in range(len(df))]
df2['Nr dokumentu zewnętrznego'] = df2['Nr dokumentu']
df2['Reservation No.'] = df['Tytuł'].apply(find_reservation).replace(' ', '')
df2['Kwota'] = -df['Kwota']
df2['Wiadomość dla odbiorcy'] = df['Tytuł']
df2['Opis'] = df2['Wiadomość dla odbiorcy'].apply(lambda x: x[:100] if isinstance(x, str) else ' ')
df2['Informacje o płatniku'] = df['Strona transakcji'].apply(lambda x: x[:50] if isinstance(x, str) else ' ')
df2['Typ konta przeciwst.'] = 'Konto bankowe'
df2['Nr konta przeciwst.'] = df['Rachunek'].apply(lambda x: get_acc_number(x)[1])
df2[['Typ konta', 'Typ dokumentu']] = df2.apply(lambda row: pd.Series(get_payment_side(row)), axis=1)

KeyError: 'Wiadomość dla odbiorcy'

In [29]:
# to excel 
df2.to_excel('import_output.xlsx', index=False)

In [30]:
# save to json the latest START_LINE + len(df) + 1
with open('last_line.txt', 'w') as f:
    f.write(str(START_LINE + len(df) + 1))